In [ ]:
import pandas as pd
import numpy as np
#import py_pcha
#from py_pcha import PCHA
import matplotlib.pyplot as plt
from numpy.matlib import repmat

In [ ]:
data=pd.read_csv("/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/us_states_covid19_daily.csv")
#data

In [ ]:
data=pd.read_csv("/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/us_states_covid19_daily.csv")
#data

In [ ]:
data_time = data[['date','state','positive','negative','totalTestResults','recovered', 'death']]
data_time = data_time.dropna()
data_time = data_time.reset_index()
data_time

In [ ]:
data_time['state'][4]

In [ ]:
states = ['CO', 'UT', 'ND', 'NY', 'NC', 'AR', 'MS', 'CT', 'IN']
data_time_sm = data_time.loc[data_time['state'].isin(states)]
data_time_sm.shape

In [ ]:
from datetime import datetime, timedelta

# you could also import date instead of datetime and use that.
#date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8]))

for i in range(len(data_time_sm)):
    string_date = data_time["date"][i]
    string_date = str(string_date)
    data_time["date"][i] = datetime(year=int(string_date[0:4]), 
                                    month=int(string_date[4:6]),
                                   day=int(string_date[6:8]))
    data_time["date"][i] = data_time["date"][i].date()
    
data_time

In [ ]:
from datetime import datetime, timedelta

# you could also import date instead of datetime and use that.
#date = datetime(year=int(s[0:4]), month=int(s[4:6]), day=int(s[6:8]))

for i in range(len(data_time)):
    string_date = data_time["date"][i]
    string_date = str(string_date)
    data_time["date"][i] = datetime(year=int(string_date[0:4]), 
                                    month=int(string_date[4:6]),
                                   day=int(string_date[6:8]))
    data_time["date"][i] = data_time["date"][i].date()
    
data_time

In [ ]:
data_time.to_csv('/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/COVID19time.csv', index = False)


In [ ]:
data_time.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19time.csv', index = False)

Data for 50 states, DC, and 5 territories on December 6, 2020. Select states and territories with missing values were removed. 

In [ ]:
dec6 = data.loc[data['date'] == 20201206]
dec6 = dec6[['state','positive','negative','totalTestResults','recovered', 'death']]
dec6 = dec6.dropna()
dec6_state = dec6[['state']]

dec6raw = dec6

dec6 = dec6[['positive','negative','totalTestResults','recovered', 'death']]
dec6 = dec6.T
#dec6_state
dec6_variables = pd.DataFrame([dec6.index]).T
#dec6raw
dec6

In [ ]:
dec6raw.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19states.csv', index = False)

In [ ]:
dec6b = data.loc[data['date'] == 20201206]
dec6b = dec6b[['positive','negative','totalTestResults','recovered', 'death', 
              'probableCases','hospitalizedCurrently',
             'negativeRegularScore','positiveScore','deathIncrease',
              'totalTestsAntibody','dataQualityGrade', 'deathProbable']]
dec6b = dec6b.dropna()
dec6b = dec6b[:4]
dec6b.T

In [ ]:
dec6.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [ ]:
dec6b.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/COVID19BIG.csv', index = False)

In [ ]:
dec6.to_csv('/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/COVID19.csv', index = False)

In [ ]:
#dec6_wo_st = dec6[['positive','negative','totalTestResults','recovered', 'death']]
dec6_wo_st = dec6.to_numpy()
dec6_wo_st

In [ ]:
XC, S, C, SSE, varexpl = PCHA(dec6_wo_st, noc=3, I = None, delta=0.1)

In [ ]:
XC

In [ ]:
def furthest_sum(K, noc, i, exclude=[]):
    
    def max_ind_val(l):
        return max(zip(range(len(l)), l), key=lambda x: x[1])
    
    I, J = K.shape
    #print(I,J)
    index = np.array(range(J))
    index[exclude] = 0
    index[i] = -1
    ind_t = i
    sum_dist = np.zeros((1, J), np.complex128)

    if J > noc * I:
        Kt = K
        Kt2 = np.sum(Kt**2, axis=0)
        for k in range(1, noc + 11):
            if k > noc - 1:
                Kq = np.dot(Kt[:, i[0]], Kt)
                #Kt_shape = Kt.shape
                #Kts = Kt[:, i[0]]
                #Ktsp = Kts.shape
                #Kq_shape = Kq.shape
                #print(Kt_shape,Ktsp,Kq_shape)
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            Kq = np.dot(Kt[:, ind_t].T, Kt)
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * Kq + Kt2[ind_t])
            #print(Kt2)
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1
    else:
        if I != J or np.sum(K - K.T) != 0:  # Generate kernel if K not one
            Kt = K
            K = np.dot(Kt.T, Kt)
            K = np.lib.scimath.sqrt(
                repmat(np.diag(K), J, 1) - 2 * K + \
                repmat(np.mat(np.diag(K)).T, 1, J)
            )

        Kt2 = np.diag(K)  # Horizontal
        for k in range(1, noc + 11):
            if k > noc - 1:
                sum_dist -= np.lib.scimath.sqrt(Kt2 - 2 * K[i[0], :] + Kt2[i[0]])
                index[i[0]] = i[0]
                i = i[1:]
            t = np.where(index != -1)[0]
            sum_dist += np.lib.scimath.sqrt(Kt2 - 2 * K[ind_t, :] + Kt2[ind_t])
            ind, val = max_ind_val(sum_dist[:, t][0].real)
            ind_t = t[ind]
            i.append(ind_t)
            index[ind_t] = -1

    return i


In [ ]:
"""Principal Convex Hull Analysis (PCHA) / Archetypal Analysis."""

import numpy as np
from scipy.sparse import csr_matrix
from datetime import datetime as dt
import time

#from py_pcha.furthest_sum import furthest_sum


def PCHAm(X, noc, I=None, U=None, delta=0, verbose=False, conv_crit=1E-6, maxiter=500):
    """Return archetypes of dataset.
    Note: Commonly data is formatted to have shape (examples, dimensions).
    This function takes input and returns output of the transposed shape,
    (dimensions, examples).
    Parameters
    ----------
    X : numpy.2darray
        Data matrix in which to find archetypes
    noc : int
        Number of archetypes to find
    I : 1d-array
        Entries of X to use for dictionary in C (optional)
    U : 1d-array
        Entries of X to model in S (optional)
    Output
    ------
    XC : numpy.2darray
        I x noc feature matrix (i.e. XC=X[:,I]*C forming the archetypes)
    S : numpy.2darray
        noc x length(U) matrix, S>=0 |S_j|_1=1
    C : numpy.2darray
        noc x length(U) matrix, S>=0 |S_j|_1=1
    SSE : float
        Sum of Squared Errors
    varexlp : float
        Percent variation explained by the model
    """
    def S_update(S, XCtX, CtXtXC, muS, SST, SSE, niter):
        """Update S for one iteration of the algorithm."""
        noc, J = S.shape
        e = np.ones((noc, 1))
        #print(e.ndim)
        for k in range(niter):
            SSE_old = SSE
            #print(SSE_old)
            g = (np.dot(CtXtXC, S) - XCtX) / (SST / J)
            #print(g)
            g = g - e * np.sum(g.A * S.A, axis=0)
            #print(g)

            S_old = S
            #print(S_old)
            while True:
                S = (S_old - g * muS).clip(min=0)
                S = S / np.dot(e, np.sum(S, axis=0))
                #print(S)
                SSt = S * S.T
                #print(SSt)
                SSE = SST - 2 * np.sum(XCtX.A * S.A) + np.sum(CtXtXC.A * SSt.A)
                #print(SSE)
                #print(SSE_old)
                if SSE <= SSE_old * (1 + 1e-9):
                    muS = muS * 1.2
                    #print(muS)
                    break
                else:
                    muS = muS / 2

        return S, SSE, muS, SSt

    def C_update(X, XSt, XC, SSt, C, delta, muC, mualpha, SST, SSE, niter=1):
        """Update C for one iteration of the algorithm."""
        J, nos = C.shape

        if delta != 0:
            alphaC = np.sum(C, axis=0).A[0]
            C = np.dot(C, np.diag(1 / alphaC))

        #print(C)
        e = np.ones((J, 1))
        XtXSt = np.dot(X.T, XSt)
        #print(XtXSt)

        for k in range(niter):

            # Update C
            SSE_old = SSE
            #print(SSE_old)
            g = (np.dot(X.T, np.dot(XC, SSt)) - XtXSt) / SST
            #print(g)

            if delta != 0:
                g = np.dot(g, np.diag(alphaC))
            g = g.A - e * np.sum(g.A * C.A, axis=0)
            #print(g)
            
            C_old = C
            while True:
                C = (C_old - muC * g).clip(min=0)
                #print(C)
                nC = np.sum(C, axis=0) + np.finfo(float).eps
                C = np.dot(C, np.diag(1 / nC.A[0]))
                #print(C)
                
                if delta != 0:
                    Ct = C * np.diag(alphaC)
                else:
                    Ct = C

                XC = np.dot(X, Ct)
                CtXtXC = np.dot(XC.T, XC)
                #print(CtXtXC)
                SSE = SST - 2 * np.sum(XC.A * XSt.A) + np.sum(CtXtXC.A * SSt.A)
                #print(SSE)
                
                if SSE <= SSE_old * (1 + 1e-9):
                    muC = muC * 1.2
                    break
                else:
                    muC = muC / 2

            # Update alphaC
            SSE_old = SSE
            if delta != 0:
                g = (np.diag(CtXtXC * SSt).T / alphaC - np.sum(C.A * XtXSt.A)) / (SST * J)
                #print(g)
                alphaC_old = alphaC
                while True:
                    alphaC = alphaC_old - mualpha * g
                    alphaC[alphaC < 1 - delta] = 1 - delta
                    alphaC[alphaC > 1 + delta] = 1 + delta

                    XCt = np.dot(XC, np.diag(alphaC / alphaC_old))
                    CtXtXC = np.dot(XCt.T, XCt)
                    SSE = SST - 2 * np.sum(XCt.A * XSt.A) + np.sum(CtXtXC.A * SSt.A)
                    #print(SSE)
                    
                    if SSE <= SSE_old * (1 + 1e-9):
                        mualpha = mualpha * 1.2
                        XC = XCt
                        break
                    else:
                        mualpha = mualpha / 2

        if delta != 0:
            C = C * np.diag(alphaC)

        return C, SSE, muC, mualpha, CtXtXC, XC

    N, M = X.shape
    

    if I is None:
        I = range(M)
    if U is None:
        U = range(M)
        
    #print(X[:,U])

    SST = np.sum(X[:, U] * X[:, U])
    #print(SST)
    
    #print(X[:,I])

    #Initialize C
    try:
       i = furthest_sum(X[:, I], noc, [int(np.ceil(len(I) * np.random.rand()))])
    except IndexError:
        class InitializationException(Exception): pass
        raise InitializationException("Initialization does not converge. Too few examples in dataset.")

    #i = [1,2,0]
        
    j = range(noc)
    C = csr_matrix((np.ones(len(i)), (i, j)), shape=(len(I), noc)).todense()
    #print(C)
    
    XC = np.dot(X[:, I], C)
    #print(XC)

    muS, muC, mualpha = 1, 1, 1

    # Initialise S
    XCtX = np.dot(XC.T, X[:, U])
    #print(XCtX)
    CtXtXC = np.dot(XC.T, XC)
    #print(CtXtXC)
    S = -np.log(np.random.random((noc, len(U))))
    #S = -np.log((np.ones((3,48))-0.5))
    #print(S)
    S = S / np.dot(np.ones((noc, 1)), np.mat(np.sum(S, axis=0)))
    #print(S)
    SSt = np.dot(S, S.T)
    SSE = SST - 2 * np.sum(XCtX.A * S.A) + np.sum(CtXtXC.A * SSt.A)
    #print(SSE)
    S, SSE, muS, SSt = S_update(S, XCtX, CtXtXC, muS, SST, SSE, 25)
    #print(S, SSE, muS, SSt)

    # Set PCHA parameters
    iter_ = 0
    dSSE = np.inf
    t1 = dt.now()
    varexpl = (SST - SSE) / SST
    #print(varexpl)

    if verbose:
        print('\nPrincipal Convex Hull Analysis / Archetypal Analysis')
        print('A ' + str(noc) + ' component model will be fitted')
        print('To stop algorithm press control C\n')

    dheader = '%10s | %10s | %10s | %10s | %10s | %10s | %10s | %10s' % ('Iteration', 'Expl. var.', 'Cost func.', 'Delta SSEf.', 'muC', 'mualpha', 'muS', ' Time(s)   ')
    dline = '-----------+------------+------------+-------------+------------+------------+------------+------------+'

    while np.abs(dSSE) >= conv_crit * np.abs(SSE) and iter_ < maxiter and varexpl < 0.9999:
        if verbose and iter_ % 100 == 0:
            print(dline)
            print(dheader)
            print(dline)
        told = t1
        iter_ += 1
        SSE_old = SSE

        # C (and alpha) update
        XSt = np.dot(X[:, U], S.T)
        #print(XSt)
        C, SSE, muC, mualpha, CtXtXC, XC = C_update(
            X[:, I], XSt, XC, SSt, C, delta, muC, mualpha, SST, SSE, 10
        )
        #print(C,SSE,muC,mualpha,CtXtXC,XC)

        # S update
        XCtX = np.dot(XC.T, X[:, U])
        #print(XCtX)
        S, SSE, muS, SSt = S_update(
            S, XCtX, CtXtXC, muS, SST, SSE, 10
        )
        #print(S,SSE,muS,SSt)

        # Evaluate and display iteration
        dSSE = SSE_old - SSE
        #print(dSSE)
        t1 = dt.now()
        if iter_ % 1 == 0:
            time.sleep(0.000001)
            varexpl = (SST - SSE) / SST
            if verbose:
                print('%10.0f | %10.4f | %10.4e | %10.4e | %10.4e | %10.4e | %10.4e | %10.4f \n' % (iter_, varexpl, SSE, dSSE/np.abs(SSE), muC, mualpha, muS, (t1-told).seconds))

    # Display final iteration
    varexpl = (SST - SSE) / SST
    #print(varexpl)
    if verbose:
        print(dline)
        print(dline)
        print('%10.0f | %10.4f | %10.4e | %10.4e | %10.4e | %10.4e | %10.4e | %10.4f \n' % (iter_, varexpl, SSE, dSSE/np.abs(SSE), muC, mualpha, muS, (t1-told).seconds))

    #print(S)
    #print(C)
    #print(XC)
        
    # Sort components according to importance
    ind, vals = zip(
        *sorted(enumerate(np.sum(S, axis=1)), key=lambda x: x[0], reverse=1)
    )
    S = S[ind, :]
    C = C[:, ind]
    XC = XC[:, ind]

    return XC, S, C, SSE, varexpl

XC2, S2, C2, SSE2, varexpl2 = PCHAm(dec6_wo_st, noc=3, I = None, delta=0)

In [ ]:
S_matrix_pd = pd.DataFrame(S2.T)

In [ ]:
dec6_state = dec6_state.reset_index()
dec6_state = dec6_state['state']
dec6_state
S_pd = pd.concat([dec6_state, S_matrix_pd], axis=1)
S_pd
#S_pd = S_pd.set_index('state')

In [ ]:
S_pd.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/SdataFrame.csv', index = False)

In [ ]:
dec6_variables = dec6_variables.rename(columns={0: "var"})
dec6_variables

In [ ]:
XC2_pd = pd.DataFrame(XC2)
XC2_pd = pd.concat([dec6_variables,XC2_pd], axis = 1)
#XC2_pd = XC2_pd.set_index("var")
XC2_pd

In [ ]:
XC2_pd.to_csv('/Users/markomiholjcic/Documents/GitHub/ArchetypalAnalysis/data/XC.csv', index = False)

In [ ]:
XC, S, C, SSE, varexpl = PCHA(dec6_wo_st, noc=3, I = None, delta=0)

In [ ]:
S[:,1]

In [ ]:
ans = 0

a = XC[:,0] - XC2[:,1]
ans = ans + a

ans2 = 0

a = XC[:,1] - XC2[:,2]
ans2 = ans2 + a

ans3 = 0

a = XC[:,2] - XC2[:,0]
ans = ans3 + a
    
print(ans, ans2, ans3)

In [ ]:
ans = 0

for i in range(len(S)):
    a = S[i] - S2[i]
    ans = ans + a
    
print(ans)

In [ ]:
ans = 0

for i in range(len(C)):
    a = C[i] - C2[i]
    ans = ans + a
    
print(ans)

In [ ]:
fifa=pd.read_csv("/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/FIFA.csv")
fifa = fifa[['Name','Age', 'Overall', 'Position', 'Crossing','Finishing','HeadingAccuracy',
            'ShortPassing','Volleys','Dribbling','Curve','FKAccuracy','LongPassing',
            'BallControl','Acceleration','SprintSpeed','Agility','Reactions','Balance',
            'ShotPower','Jumping','Stamina','Strength','LongShots','Aggression',
            'Interceptions','Positioning','Vision','Penalties','Composure','Marking',
            'StandingTackle','SlidingTackle','GKDiving','GKHandling','GKKicking',
            'GKPositioning','GKReflexes']]
fifa.dtypes

In [ ]:
#fifa = fifa.drop(labels=['Name', 'Position'], axis=1)
fifa = pd.DataFrame(fifa)
fifa = fifa.dropna()
#fifa_num = fifa.to_numpy

In [ ]:
fifa.to_csv('/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/fifaRatings.csv', index = False)

In [ ]:
fifa_s = fifa.head(20)
fifa_s

In [ ]:
fifa_s.to_csv('/Users/u0984549/Documents/GitHub/ArchetypalAnalysis/data/fifaRatingsSmall.csv', index = False)